In [105]:
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import time
import requests
import json
from config import omdb_key, username, password
from pprint import pprint

# Extract and Transform

* IMDB's Top 250 movies was scraped from https://www.imdb.com/chart/top/?ref_=nv_mv_250 and converted into a dataframe
* Netflix's and Disney Plus' databases were found from Kaggle while the urls are from https://reelgood.com/movies/
* Prime Video's and Hulu's databases were from https://reelgood.com/movies/

## IMDB

In [48]:
# Extract list of top 250 movies according to IMDB
url = "https://www.imdb.com/chart/top/?ref_=nv_mv_250"
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [49]:
movies = soup.find('tbody', class_="lister-list").find_all('td', class_="titleColumn")
movie_ratings = soup.find('tbody', class_="lister-list").find_all('td', class_="imdbRating")

ratings = []

for rating in movie_ratings:
    ratings.append(rating.text.strip())

In [50]:
i = 0
rank = []
titles = []
year = []
ids = []
for movie in movies:
    i += 1
    print(str(i) + ". " + movie.a.text +" ("+ movie.span.text.replace('(','').replace(')','') + ")")
    rank.append(i)
    titles.append(movie.a.text)
    year.append(movie.span.text.replace('(','').replace(')',''))
    ids.append(movie.a['href'].split('/')[2])


1. The Shawshank Redemption (1994)
2. The Godfather (1972)
3. The Godfather: Part II (1974)
4. The Dark Knight (2008)
5. 12 Angry Men (1957)
6. Schindler's List (1993)
7. The Lord of the Rings: The Return of the King (2003)
8. Pulp Fiction (1994)
9. The Good, the Bad and the Ugly (1966)
10. The Lord of the Rings: The Fellowship of the Ring (2001)
11. Fight Club (1999)
12. Forrest Gump (1994)
13. Inception (2010)
14. Star Wars: Episode V - The Empire Strikes Back (1980)
15. The Lord of the Rings: The Two Towers (2002)
16. The Matrix (1999)
17. Goodfellas (1990)
18. One Flew Over the Cuckoo's Nest (1975)
19. Seven Samurai (1954)
20. Se7en (1995)
21. Life Is Beautiful (1997)
22. City of God (2002)
23. The Silence of the Lambs (1991)
24. It's a Wonderful Life (1946)
25. Star Wars: Episode IV - A New Hope (1977)
26. Saving Private Ryan (1998)
27. Parasite (2019)
28. Spirited Away (2001)
29. The Green Mile (1999)
30. Interstellar (2014)
31. Léon: The Professional (1994)
32. The Usual Suspect

In [57]:
# Create IMDB df
imdb_df = pd.DataFrame({"rank": rank, "id": ids, "title": titles, "year": year, "rating": ratings})
imdb_df

,rank,id,title,year,rating
0,1,tt0111161,The Shawshank Redemption,1994,9.2
1,2,tt0068646,The Godfather,1972,9.1
2,3,tt0071562,The Godfather: Part II,1974,9.0
3,4,tt0468569,The Dark Knight,2008,9.0
4,5,tt0050083,12 Angry Men,1957,8.9
...,...,...,...,...,...
245,246,tt0103639,Aladdin,1992,8.0
246,247,tt0083922,Fanny and Alexander,1982,8.0
247,248,tt0169102,Lagaan: Once Upon a Time in India,2001,8.0
248,249,tt5323662,A Silent Voice: The Movie,2016,8.0


In [53]:
browser.quit()

In [107]:
# Extract director and mpaa_rating
url = "http://www.omdbapi.com/?i="
api_key = "&apikey=" + omdb_key

In [108]:
mpaa_ratings = []
genres = []
directors = []

for index, row in imdb_df.iterrows():
    response = requests.get(url + row[1] + api_key)
    data = response.json()
    mpaa_ratings.append(data['Rated'])
    genres.append(data['Genre'])
    directors.append(data['Director'])
    
imdb_df['mpaa_rating'] = mpaa_ratings
imdb_df['genre'] = genres
imdb_df['director'] = directors

imdb_df
    

,rank,id,title,year,rating,mpaa_rating,genre,director
0,1,tt0111161,The Shawshank Redemption,1994,9.2,R,Drama,Frank Darabont
1,2,tt0068646,The Godfather,1972,9.1,R,"Crime, Drama",Francis Ford Coppola
2,3,tt0071562,The Godfather: Part II,1974,9.0,R,"Crime, Drama",Francis Ford Coppola
3,4,tt0468569,The Dark Knight,2008,9.0,PG-13,"Action, Crime, Drama, Thriller",Christopher Nolan
4,5,tt0050083,12 Angry Men,1957,8.9,Approved,Drama,Sidney Lumet
...,...,...,...,...,...,...,...,...
245,246,tt0103639,Aladdin,1992,8.0,G,"Animation, Adventure, Comedy, Family, Fantasy,...","Ron Clements, John Musker"
246,247,tt0083922,Fanny and Alexander,1982,8.0,R,Drama,Ingmar Bergman
247,248,tt0169102,Lagaan: Once Upon a Time in India,2001,8.0,PG,"Adventure, Drama, Musical, Sport",Ashutosh Gowariker
248,249,tt5323662,A Silent Voice: The Movie,2016,8.0,Not Rated,"Animation, Drama, Family, Romance",Naoko Yamada


## Netflix

In [94]:
# Import netflix raw file from Kaggle
netflix_raw_csv = "Resources/netflix_titles_raw.csv"
netflix_df = pd.read_csv(netflix_raw_csv)
netflix_df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,NaN,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,NaN,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."
3,80058654,TV Show,Transformers: Robots in Disguise,NaN,"Will Friedle, Darren Criss, Constance Zimmer, ...",United States,"September 8, 2018",2016,TV-Y7,1 Season,Kids' TV,When a prison ship crash unleashes hundreds of...
4,80125979,Movie,#realityhigh,Fernando Lebrija,"Nesta Cooper, Kate Walsh, John Michael Higgins...",United States,"September 8, 2017",2017,TV-14,99 min,Comedies,When nerdy high schooler Dani finally attracts...


In [113]:
# Match IMDB and netflix movies
imdb_netflix = pd.merge(imdb_df, netflix_df, on="title")
imdb_netflix = imdb_netflix[["rank", "id", "title", "rating_x", "year", "director_x", "genre", "mpaa_rating"]]
imdb_netflix = imdb_netflix.rename(columns={"rating_x":"rating", "director_x": "director"})
imdb_netflix["service"] = "netflix"
imdb_netflix = imdb_netflix.drop_duplicates(subset="id", keep="first")
imdb_netflix

,rank,id,title,rating,year,director,genre,mpaa_rating,service
0,6,tt0108052,Schindler's List,8.9,1993,Steven Spielberg,"Biography, Drama, History",R,netflix
1,7,tt0167260,The Lord of the Rings: The Return of the King,8.9,2003,Peter Jackson,"Adventure, Drama, Fantasy",PG-13,netflix
2,8,tt0110912,Pulp Fiction,8.8,1994,Quentin Tarantino,"Crime, Drama",R,netflix
3,13,tt1375666,Inception,8.7,2010,Christopher Nolan,"Action, Adventure, Sci-Fi, Thriller",PG-13,netflix
4,15,tt0167261,The Lord of the Rings: The Two Towers,8.7,2002,Peter Jackson,"Adventure, Drama, Fantasy",PG-13,netflix
5,16,tt0133093,The Matrix,8.6,1999,"Lana Wachowski, Lilly Wachowski","Action, Sci-Fi",R,netflix
6,22,tt0317248,City of God,8.6,2002,"Fernando Meirelles, Kátia Lund(co-director)","Crime, Drama",R,netflix
7,35,tt0120586,American History X,8.5,1998,Tony Kaye,Drama,R,netflix
8,43,tt0407887,The Departed,8.5,2006,Martin Scorsese,"Crime, Drama, Thriller",R,netflix
9,47,tt0064116,Once Upon a Time in the West,8.5,1968,Sergio Leone,Western,PG-13,netflix


In [96]:
# Extract urls
url = "https://reelgood.com/movies/source/netflix?filter-imdb_start=8"
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [97]:
for x in range(1, 5):
    try:
        browser.find_by_css('button.css-qbp45y.eyx6tna2').click()
        time.sleep(3)
        print("Page: " + str(x))
    except:
        print("Last Page Reached")


Page: 1
Page: 2
Page: 3
Last Page Reached


In [98]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
movies = soup.find('table', class_="css-1179hly").find_all('td', class_="css-1u7zfla e126mwsw1")

In [99]:
titles = []
nf_urls = []

for movie in movies:
    titles.append(movie.a.text)
    rg_url = movie.find('meta', itemprop="url")['content']
    response = requests.get(rg_url)
    rg_soup = BeautifulSoup(response.text, 'html.parser')
    nf_urls.append(rg_soup.find('div',title="Stream on Netflix").a['href'].split('?')[0])
    print(movie.a.text +" - "+ rg_soup.find('div',title="Stream on Netflix").a['href'].split('?')[0])

Inception - https://www.netflix.com/watch/70131314
The Matrix - https://www.netflix.com/watch/20557937
Avengers: Infinity War - https://www.netflix.com/watch/80219127
Back to the Future - https://www.netflix.com/watch/60010110
The Good, the Bad and the Ugly - https://www.netflix.com/watch/553500
Spider-Man: Into the Spider-Verse - https://www.netflix.com/watch/81002747
Django Unchained - https://www.netflix.com/watch/70230640
The Pianist - https://www.netflix.com/watch/60025061
Raiders of the Lost Ark - https://www.netflix.com/watch/60011649
Inglourious Basterds - https://www.netflix.com/watch/70108777
Taxi Driver - https://www.netflix.com/watch/18907685
3 Idiots - https://www.netflix.com/watch/70121522
Pan's Labyrinth - https://www.netflix.com/watch/70050507
Room - https://www.netflix.com/watch/80073823
Monty Python and the Holy Grail - https://www.netflix.com/watch/771476
Groundhog Day - https://www.netflix.com/watch/563104
Indiana Jones and the Last Crusade - https://www.netflix.com

Tim Minchin and the Heritage Orchestra: Live at the Royal Albert Hall - https://www.netflix.com/watch/70298389
Dave Chappelle: Equanimity - https://www.netflix.com/watch/80230402
Our Planet: Behind The Scenes - https://www.netflix.com/watch/81082125
Muramba - https://www.netflix.com/watch/80223049
Wrong Side Raju - https://www.netflix.com/watch/80208413
Tikli and Laxmi Bomb - https://www.netflix.com/watch/81002213
My Little Pony: Best Gift Ever - https://www.netflix.com/watch/81018860
One Heart: The A.R. Rahman Concert Film - https://www.netflix.com/watch/80216270
Gardeners of Eden - https://www.netflix.com/watch/80052703
Shuddhi - https://www.netflix.com/watch/80188777
Tim Minchin: So F**king Rock Live - https://www.netflix.com/watch/81045530
Shonar Pahar - https://www.netflix.com/watch/81071868
We Are Moluccans - https://www.netflix.com/watch/81016323
Hikaru Utada Laughter in the Dark Tour 2018 - https://www.netflix.com/watch/81092491
Bill Hicks: One Night Stand - https://www.netflix

In [ ]:
movie_items = soup.find('table', class_="css-1179hly").find_all('tr', class_="css-o6sgwe")

years = []

for movie in movie_items:
    td_rating = movie.find('td', class_="css-1u11l3y")
    years.append(td_rating.text)
    print(td_rating.text)

In [ ]:
netflix_url_df = pd.DataFrame({"title": titles, "year": years, "netflix_url": nf_urls})
netflix_url_df

In [ ]:
browser.quit()

In [115]:
# Add urls
netflix_newdf = pd.merge(imdb_netflix, netflix_url_df, on='title', how='left' )
netflix_newdf = netflix_newdf[["rank", "id", "title", "rating", "year_x", "director", "genre", "mpaa_rating", "service", "netflix_url"]]
netflix_newdf = netflix_newdf.rename(columns={"year_x":"year", "netflix_url":"url"})
netflix_newdf["url"].fillna("Not Available", inplace = True)
netflix_newdf

,rank,id,title,rating,year,director,genre,mpaa_rating,service,url
0,6,tt0108052,Schindler's List,8.9,1993,Steven Spielberg,"Biography, Drama, History",R,netflix,Not Available
1,7,tt0167260,The Lord of the Rings: The Return of the King,8.9,2003,Peter Jackson,"Adventure, Drama, Fantasy",PG-13,netflix,Not Available
2,8,tt0110912,Pulp Fiction,8.8,1994,Quentin Tarantino,"Crime, Drama",R,netflix,Not Available
3,13,tt1375666,Inception,8.7,2010,Christopher Nolan,"Action, Adventure, Sci-Fi, Thriller",PG-13,netflix,https://www.netflix.com/watch/70131314
4,15,tt0167261,The Lord of the Rings: The Two Towers,8.7,2002,Peter Jackson,"Adventure, Drama, Fantasy",PG-13,netflix,Not Available
5,16,tt0133093,The Matrix,8.6,1999,"Lana Wachowski, Lilly Wachowski","Action, Sci-Fi",R,netflix,https://www.netflix.com/watch/20557937
6,22,tt0317248,City of God,8.6,2002,"Fernando Meirelles, Kátia Lund(co-director)","Crime, Drama",R,netflix,Not Available
7,35,tt0120586,American History X,8.5,1998,Tony Kaye,Drama,R,netflix,Not Available
8,43,tt0407887,The Departed,8.5,2006,Martin Scorsese,"Crime, Drama, Thriller",R,netflix,Not Available
9,47,tt0064116,Once Upon a Time in the West,8.5,1968,Sergio Leone,Western,PG-13,netflix,Not Available


## Disney Plus

In [139]:
# Import csv raw file from Kaggle
csv_file = "Resources/disney_plus_raw.csv"
disney_df = pd.read_csv(csv_file)
disney_df.head()


,imdb_id,title,plot,type,rated,year,released_at,added_at,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes
0,tt0147800,10 Things I Hate About You,"A pretty, popular teenager can't go out on a d...",movie,PG-13,1999,31 Mar 1999,"November 12, 2019",97 min,"Comedy, Drama, Romance",Gil Junger,"Karen McCullah, Kirsten Smith","Heath Ledger, Julia Stiles, Joseph Gordon-Levi...","English, French",USA,2 wins & 13 nominations.,70.0,7.3,"283,945"
1,tt7019028,101 Dalmatian Street,This series follows the lives of Delilah and D...,series,NaN,2018–,25 Mar 2019,"February 28, 2020",NaN,"Animation, Comedy, Family",NaN,NaN,"Josh Brener, Michaela Dietz, Bert Davis, Abiga...",English,"UK, USA, Canada",NaN,NaN,6.2,124
2,tt0115433,101 Dalmatians,An evil high-fashion designer plots to steal D...,movie,G,1996,27 Nov 1996,"November 12, 2019",103 min,"Adventure, Comedy, Crime, Family",Stephen Herek,"Dodie Smith (novel), John Hughes (screenplay)","Glenn Close, Jeff Daniels, Joely Richardson, J...","English, Spanish","USA, UK",Nominated for 1 Golden Globe. Another 3 wins &...,49.0,5.7,"97,785"
3,tt0324941,101 Dalmatians 2: Patch's London Adventure,"Being one of 101 takes its toll on Patch, who ...",movie,G,2002,21 Jan 2003,"November 12, 2019",74 min,"Animation, Adventure, Comedy, Family, Musical","Jim Kammerud, Brian Smith","Jim Kammerud (story), Dan Root (story), Garret...","Barry Bostwick, Jason Alexander, Martin Short,...",English,USA,5 wins & 10 nominations.,NaN,5.8,"7,434"
4,tt0211181,102 Dalmatians,Cruella DeVil gets out of prison and goes afte...,movie,G,2000,22 Nov 2000,"November 12, 2019",100 min,"Adventure, Comedy, Family",Kevin Lima,"Dodie Smith (novel), Kristen Buckley (story), ...","Glenn Close, Gérard Depardieu, Ioan Gruffudd, ...",English,"USA, UK",Nominated for 1 Oscar. Another 1 win & 7 nomin...,35.0,4.9,"33,444"


In [65]:
# Clean df
disney_df = disney_df[["title", "year"]]
print(len(disney_df))
disney_df

992


,title,year
0,10 Things I Hate About You,1999
1,101 Dalmatian Street,2018–
2,101 Dalmatians,1996
3,101 Dalmatians 2: Patch's London Adventure,2002
4,102 Dalmatians,2000
...,...,...
987,Zapped,2014
988,Zenon: Girl of the 21st Century,1999
989,Zenon: The Zequel,2001
990,Zenon: Z3,2004


In [118]:
# Match IMDB with Disney file
imdb_disney = pd.merge(imdb_df, disney_df, on="title")
imdb_disney = imdb_disney[["rank", "id", "title", "rating", "year_x", "director", "genre", "mpaa_rating"]]
imdb_disney = imdb_disney.rename(columns={"year_x":"year"})
imdb_disney = imdb_disney.drop_duplicates(subset="id", keep="first")
imdb_disney["service"] = "disney plus"
imdb_disney

,rank,id,title,rating,year,director,genre,mpaa_rating,service
0,14,tt0080684,Star Wars: Episode V - The Empire Strikes Back,8.7,1980,Irvin Kershner,"Action, Adventure, Fantasy, Sci-Fi",PG,disney plus
1,25,tt0076759,Star Wars: Episode IV - A New Hope,8.6,1977,George Lucas,"Action, Adventure, Fantasy, Sci-Fi",PG,disney plus
2,34,tt0110357,The Lion King,8.5,1994,"Roger Allers, Rob Minkoff","Animation, Adventure, Drama, Family, Musical",G,disney plus
4,63,tt0910970,WALL·E,8.4,2008,Andrew Stanton,"Animation, Adventure, Family, Sci-Fi",G,disney plus
5,70,tt4154796,Avengers: Endgame,8.3,2019,"Anthony Russo, Joe Russo","Action, Adventure, Drama, Sci-Fi",PG-13,disney plus
6,75,tt2380307,Coco,8.3,2017,"Lee Unkrich, Adrian Molina(co-director)","Animation, Adventure, Family, Fantasy, Music, ...",PG,disney plus
7,81,tt0114709,Toy Story,8.3,1995,John Lasseter,"Animation, Adventure, Comedy, Family, Fantasy",G,disney plus
8,85,tt0086190,Star Wars: Episode VI - Return of the Jedi,8.3,1983,Richard Marquand,"Action, Adventure, Fantasy, Sci-Fi",PG,disney plus
9,103,tt0012349,The Kid,8.2,1921,Charles Chaplin,"Comedy, Drama, Family",Passed,disney plus
10,111,tt0435761,Toy Story 3,8.2,2010,Lee Unkrich,"Animation, Adventure, Comedy, Family, Fantasy",G,disney plus


In [68]:
# Extract Disney URLs
url = "https://reelgood.com/movies/source/disney_plus?filter-imdb_start=8"
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [69]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
movies = soup.find('table', class_="css-1179hly").find_all('td', class_="css-1u7zfla e126mwsw1")

In [70]:
titles = []
dp_urls = []

for movie in movies:
    titles.append(movie.a.text)
    rg_url = movie.find('meta', itemprop="url")['content']
    response = requests.get(rg_url)
    rg_soup = BeautifulSoup(response.text, 'html.parser')
    dp_urls.append(rg_soup.find('div',title="Stream on Disney+").a['href'].split('?')[0])
    print(movie.a.text +" - "+ rg_soup.find('div',title="Stream on Disney+").a['href'].split('?')[0])

Star Wars: A New Hope - https://www.disneyplus.com/video/f4add311-5f2a-4d79-a6c3-588a1765b7d9
Star Wars: The Empire Strikes Back - https://www.disneyplus.com/video/90ff6e09-afd3-4d46-a2cb-80e3767e67ea
The Lion King - https://www.disneyplus.com/video/87982e2b-5636-41a2-9d9a-5bc77cacb7d7
The Avengers - https://www.disneyplus.com/video/c05ea526-1378-4e4b-a38e-a29968cc0def
Avengers: Endgame - https://www.disneyplus.com/video/aa2c93d5-6e09-4e2b-84ae-9f0773cc0238
Toy Story - https://www.disneyplus.com/video/eccdbdc5-7767-448e-be14-74647b128ca9
WALL·E - https://www.disneyplus.com/video/825ef1dd-24fe-4f32-90a0-adc5e94a8023
Up - https://www.disneyplus.com/video/6a45dca1-b196-4891-bd98-f8dd5098518f
Guardians of the Galaxy - https://www.disneyplus.com/video/04667710-5866-4a77-be5d-a9f7265bdb5c
Finding Nemo - https://www.disneyplus.com/video/7d1b7852-f2ce-4ccb-a73f-5d7eba132bb2
Coco - https://www.disneyplus.com/video/4ccb6027-d7ca-4b81-8c70-43ac08f92bbd
Inside Out - https://www.disneyplus.com/vide

In [71]:
movie_items = soup.find('table', class_="css-1179hly").find_all('tr', class_="css-o6sgwe")

years = []

for movie in movie_items:
    td_rating = movie.find('td', class_="css-1u11l3y")
    years.append(td_rating.text)
    print(td_rating.text)

1977
1980
1994
2012
2019
1995
2008
2009
2014
2003
2017
2015
2010
2001
1983
2016
1987
2007
1992
1991
2004
2003
1993
2018
1965
2019
1999
1983
2016
2004
2017
2019
2020
2014
2013
1953
2016
2018


In [76]:
disney_url_df = pd.DataFrame({"title": titles, "year": years, "disneyplus_url": dp_urls})
disney_url_df

disney_url_df.loc[(disney_url_df.title == "Star Wars: A New Hope"),'title']="Star Wars: Episode IV - A New Hope"
disney_url_df.loc[(disney_url_df.title == 'Star Wars: The Empire Strikes Back'),'title']='Star Wars: Episode V - The Empire Strikes Back'
disney_url_df.loc[(disney_url_df.title == 'Star Wars: Return of the Jedi'),'title']='Star Wars: Episode VI - Return of the Jedi'
disney_url_df

,title,year,disneyplus_url
0,Star Wars: Episode IV - A New Hope,1977,https://www.disneyplus.com/video/f4add311-5f2a...
1,Star Wars: Episode V - The Empire Strikes Back,1980,https://www.disneyplus.com/video/90ff6e09-afd3...
2,The Lion King,1994,https://www.disneyplus.com/video/87982e2b-5636...
3,The Avengers,2012,https://www.disneyplus.com/video/c05ea526-1378...
4,Avengers: Endgame,2019,https://www.disneyplus.com/video/aa2c93d5-6e09...
5,Toy Story,1995,https://www.disneyplus.com/video/eccdbdc5-7767...
6,WALL·E,2008,https://www.disneyplus.com/video/825ef1dd-24fe...
7,Up,2009,https://www.disneyplus.com/video/6a45dca1-b196...
8,Guardians of the Galaxy,2014,https://www.disneyplus.com/video/04667710-5866...
9,Finding Nemo,2003,https://www.disneyplus.com/video/7d1b7852-f2ce...


In [74]:
browser.quit()

In [121]:
# Add urls
disney_newdf = pd.merge(imdb_disney, disney_url_df, on='title', how='left' )
disney_newdf = disney_newdf[["rank", "id", "title", "rating", "year_x", "director", "genre", "mpaa_rating", "service", "disneyplus_url"]]
disney_newdf = disney_newdf.rename(columns={"year_x":"year", "disneyplus_url":"url"})
disney_newdf["url"].fillna("Not Available", inplace = True)
disney_newdf

,rank,id,title,rating,year,director,genre,mpaa_rating,service,url
0,14,tt0080684,Star Wars: Episode V - The Empire Strikes Back,8.7,1980,Irvin Kershner,"Action, Adventure, Fantasy, Sci-Fi",PG,disney plus,https://www.disneyplus.com/video/90ff6e09-afd3...
1,25,tt0076759,Star Wars: Episode IV - A New Hope,8.6,1977,George Lucas,"Action, Adventure, Fantasy, Sci-Fi",PG,disney plus,https://www.disneyplus.com/video/f4add311-5f2a...
2,34,tt0110357,The Lion King,8.5,1994,"Roger Allers, Rob Minkoff","Animation, Adventure, Drama, Family, Musical",G,disney plus,https://www.disneyplus.com/video/87982e2b-5636...
3,63,tt0910970,WALL·E,8.4,2008,Andrew Stanton,"Animation, Adventure, Family, Sci-Fi",G,disney plus,https://www.disneyplus.com/video/825ef1dd-24fe...
4,70,tt4154796,Avengers: Endgame,8.3,2019,"Anthony Russo, Joe Russo","Action, Adventure, Drama, Sci-Fi",PG-13,disney plus,https://www.disneyplus.com/video/aa2c93d5-6e09...
5,75,tt2380307,Coco,8.3,2017,"Lee Unkrich, Adrian Molina(co-director)","Animation, Adventure, Family, Fantasy, Music, ...",PG,disney plus,https://www.disneyplus.com/video/4ccb6027-d7ca...
6,81,tt0114709,Toy Story,8.3,1995,John Lasseter,"Animation, Adventure, Comedy, Family, Fantasy",G,disney plus,https://www.disneyplus.com/video/eccdbdc5-7767...
7,85,tt0086190,Star Wars: Episode VI - Return of the Jedi,8.3,1983,Richard Marquand,"Action, Adventure, Fantasy, Sci-Fi",PG,disney plus,https://www.disneyplus.com/video/1b9be263-68b4...
8,103,tt0012349,The Kid,8.2,1921,Charles Chaplin,"Comedy, Drama, Family",Passed,disney plus,Not Available
9,111,tt0435761,Toy Story 3,8.2,2010,Lee Unkrich,"Animation, Adventure, Comedy, Family, Fantasy",G,disney plus,https://www.disneyplus.com/video/ea032f8d-f213...


## Prime Video

In [81]:
# Extract prime video movies
url = "https://reelgood.com/movies/source/amazon?filter-imdb_start=8"
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [82]:
for x in range(1, 11):
    try:
        browser.find_by_css('button.css-qbp45y.eyx6tna2').click()
        time.sleep(3)
        print("Page: " + str(x))
    except:
        print("Last Page Reached")

html = browser.html
soup = BeautifulSoup(html, 'html.parser')

movies = soup.find('table', class_="css-1179hly").find_all('td', class_="css-1u7zfla e126mwsw1")

Page: 1
Page: 2
Page: 3
Page: 4
Page: 5
Page: 6
Page: 7
Page: 8
Last Page Reached
Last Page Reached


In [83]:
titles = []
ap_urls = []

for movie in movies:
    titles.append(movie.a.text)
    rg_url = movie.find('meta', itemprop="url")['content']
    response = requests.get(rg_url)
    rg_soup = BeautifulSoup(response.text, 'html.parser')
    ap_urls.append(rg_soup.find('div',title="Stream on Prime Video").a['href'].split('?')[0])
    print(movie.a.text +" - "+ rg_soup.find('div',title="Stream on Prime Video").a['href'].split('?')[0])

The Good, the Bad and the Ugly - https://www.amazon.com/gp/video/detail/B0011XBLUS
The Pianist - https://www.amazon.com/gp/video/detail/B084TMR3XK
The Avengers - https://www.amazon.com/gp/video/detail/B008NCSZQ8
3 Idiots - https://www.amazon.com/gp/video/detail/B07V7T25QS
Once Upon a Time in the West - https://www.amazon.com/gp/video/detail/B07YMSH52P
It's a Wonderful Life - https://www.amazon.com/gp/video/detail/B07JMZ4HVH
Downfall - https://www.amazon.com/gp/video/detail/B07TJ26FVJ
Sunset Boulevard - https://www.amazon.com/gp/video/detail/B07VRF8CMZ
The Great Escape - https://www.amazon.com/gp/video/detail/B000IZ0WQ4
Warrior - https://www.amazon.com/gp/video/detail/B008Y6QZ56
Some Like It Hot - https://www.amazon.com/gp/video/detail/B00477MY1K
The Handmaiden - https://www.amazon.com/gp/video/detail/B0751JGSCT
Rosemary's Baby - https://www.amazon.com/gp/video/detail/B07YVKJ64Q
The Kid - https://www.amazon.com/gp/video/detail/B0010WF6K6
Amores Perros - https://www.amazon.com/gp/video/d

George Carlin: Doin' it Again - https://www.amazon.com/gp/video/detail/B002PSVNTW
Father of a Soldier - https://www.amazon.com/gp/video/detail/B0786NS1G8
The Good Soldier - https://www.amazon.com/gp/video/detail/B07JJB2CCG
Bill Hicks: Revelations - https://www.amazon.com/gp/video/detail/B07N8MJWJG
Jim Jefferies: I Swear to God - https://www.amazon.com/gp/video/detail/B007Q35T8M
Jeff Dunham: Arguing with Myself - https://www.amazon.com/gp/video/detail/B07Q5CFQRN
Awaara - https://www.amazon.com/gp/video/detail/B0813ZHNGH
Chhoti Si Baat - https://www.amazon.com/gp/video/detail/B07V8LKK8N
Maanagaram - https://www.amazon.com/gp/video/detail/B071L7TL92
Disorder in the Court - https://www.amazon.com/gp/video/detail/B002KOIIVW
Coldplay: Live 2003 - https://www.amazon.com/gp/video/detail/B07L4YD35L
George Carlin: George's Best Stuff - https://www.amazon.com/gp/video/detail/B00M0CL6OU
Hearing Is Believing - https://www.amazon.com/gp/video/detail/B073162RQP
Only Old Men Are Going to Battle - http

Impact After the Crash - https://www.amazon.com/gp/video/detail/B06X6K7KMH
Lost Kites - https://www.amazon.com/gp/video/detail/B07JH5SRN3
Fools on the Hill - https://www.amazon.com/gp/video/detail/B01HSTE7E0
The Creators - https://www.amazon.com/gp/video/detail/B00TG1TFZU
Most Valuable Players - https://www.amazon.com/gp/video/detail/B07NJ749N9
The Weekend Sailor - https://www.amazon.com/gp/video/detail/B01MU7T5WD
The Coexist Comedy Tour - https://www.amazon.com/gp/video/detail/B075G42VN9
Brad Williams: Fun Size - https://www.amazon.com/gp/video/detail/B06XSQQVLH
BLUE - https://www.amazon.com/gp/video/detail/B0736DVVGC
Mumbai Pune Mumbai - https://www.amazon.com/gp/video/detail/B086TVHMQZ
Arise - https://www.amazon.com/gp/video/detail/B01DFQASK2
Grateful Dead: Dead Ahead - https://www.amazon.com/gp/video/detail/B071ZPGTDZ
Memories 677 - https://www.amazon.com/gp/video/detail/B077QTRQJN
Men Of The Cloth - https://www.amazon.com/gp/video/detail/B0732HVRXD
Stronger Than Bullets - https://

Wyeth - https://www.amazon.com/gp/video/detail/B07GWCQJ5C
8 Wheels & Some Soul Brotha' Music - https://www.amazon.com/gp/video/detail/B07K3Z8BHD
Forgotten - https://www.amazon.com/gp/video/detail/B06XCDR9Y3
Del Shores: Sordid Confessions - https://www.amazon.com/gp/video/detail/B016VEHN4Y
The Long Way Back: The Story of Todd Z-Man Zalkins - https://www.amazon.com/gp/video/detail/B07CKF68V6
Making Marines - https://www.amazon.com/gp/video/detail/B071FN4SBV
Love, Concord - https://www.amazon.com/gp/video/detail/B016LO6Z66
Slednecks 13 - https://www.amazon.com/gp/video/detail/B07VMTDNKY
Yes: Symphonic Live - https://www.amazon.com/gp/video/detail/B07JY9F3G9
Mixed Match - https://www.amazon.com/gp/video/detail/B07GY2WDKV
Elvis: The Memphis Flash - https://www.amazon.com/gp/video/detail/B07BZ154QS
Flakes & Flames - https://www.amazon.com/gp/video/detail/B06ZYMTZ57
Local Legends - https://www.amazon.com/gp/video/detail/B07LGG43TW
Life on the V: The Story of V66 - https://www.amazon.com/gp/vi

In [84]:
movie_items = soup.find('table', class_="css-1179hly").find_all('tr', class_="css-o6sgwe")

years = []

for movie in movie_items:
    td_rating = movie.find('td', class_="css-1u11l3y")
    years.append(td_rating.text)
    print(td_rating.text)

1966
2002
2012
2009
1968
1946
2004
1950
1963
2011
1959
2016
1968
1921
2000
1953
2013
1955
2008
1926
1902
1920
1984
2010
1953
2012
1962
1996
2007
2018
2004
2011
2013
1957
2019
2013
1978
2014
1971
1936
1975
1982
2011
2017
2018
2018
1982
2016
2007
1946
2007
1957
2002
1977
2013
2018
2014
2016
1995
2005
2013
2012
2017
1987
2010
2017
2015
2001
2009
1994
2009
2011
1971
1999
2007
2013
1975
1979
2004
2015
2014
2014
2017
2006
2017
2018
2017
1996
1992
1983
1999
2013
2012
2015
2013
2000
2007
2005
2014
1989
2020
1962
2000
2001
2005
2012
1992
1987
1968
2002
1972
2017
1983
1985
2014
2019
2008
1990
1964
2009
1993
2009
2006
1951
1976
2017
1936
2003
1996
2017
1973
2017
1950
1986
2018
2015
2009
2007
2017
1979
2017
1999
2017
2012
1990
1997
1959
2012
1955
1996
1988
1993
2012
1989
1953
2015
1958
2015
2011
2010
2014
2005
2009
2018
2015
1989
1984
1984
2013
2009
2003
2015
1964
1986
1972
2012
1982
1978
2015
2016
2019
2010
1956
2017
2015
1994
2001
2013
2004
2014
2006
2014
1986
2007
2004
2011
2015
2013
2008
2004


In [85]:
# Create df
prime_df = pd.DataFrame({"title": titles, "year": years, "amazon_prime_url": ap_urls})
prime_df = prime_df[["title", "year", "amazon_prime_url"]]
prime_df

,title,year,amazon_prime_url
0,"The Good, the Bad and the Ugly",1966,https://www.amazon.com/gp/video/detail/B0011XBLUS
1,The Pianist,2002,https://www.amazon.com/gp/video/detail/B084TMR3XK
2,The Avengers,2012,https://www.amazon.com/gp/video/detail/B008NCSZQ8
3,3 Idiots,2009,https://www.amazon.com/gp/video/detail/B07V7T25QS
4,Once Upon a Time in the West,1968,https://www.amazon.com/gp/video/detail/B07YMSH52P
...,...,...,...
399,Help Us Find Sunil Tripathi,2015,https://www.amazon.com/gp/video/detail/B07KL3KSGZ
400,Shadow of Afghanistan,2006,https://www.amazon.com/gp/video/detail/B007EMBQDC
401,Doolittle's Raiders: A Final Toast,2014,https://www.amazon.com/gp/video/detail/B088STPVCR
402,Check Your Body at the Door,2012,https://www.amazon.com/gp/video/detail/B07F75HVZB


In [ ]:
browser.quit()

In [122]:
# Match IMDB250 and prime
imdb_prime = pd.merge(imdb_df, prime_df, on="title")
imdb_prime = imdb_prime[["rank", "id", "title", "rating", "year_x", "amazon_prime_url", "director", "genre", "mpaa_rating"]]
imdb_prime = imdb_prime.rename(columns={"year_x":"year", "amazon_prime_url":"url"})
imdb_prime["service"] = "prime video"
imdb_prime

,rank,id,title,rating,year,url,director,genre,mpaa_rating,service
0,9,tt0060196,"The Good, the Bad and the Ugly",8.8,1966,https://www.amazon.com/gp/video/detail/B0011XBLUS,Sergio Leone,Western,R,prime video
1,24,tt0038650,It's a Wonderful Life,8.6,1946,https://www.amazon.com/gp/video/detail/B07JMZ4HVH,Frank Capra,"Drama, Family, Fantasy",Passed,prime video
2,36,tt0253474,The Pianist,8.5,2002,https://www.amazon.com/gp/video/detail/B084TMR3XK,Roman Polanski,"Biography, Drama, Music, War",R,prime video
3,47,tt0064116,Once Upon a Time in the West,8.5,1968,https://www.amazon.com/gp/video/detail/B07YMSH52P,Sergio Leone,Western,PG-13,prime video
4,77,tt0066763,Anand,8.3,1971,https://www.amazon.com/gp/video/detail/B07LDDZ163,Hrishikesh Mukherjee,"Drama, Musical",Not Rated,prime video
5,80,tt1187043,3 Idiots,8.3,2009,https://www.amazon.com/gp/video/detail/B07V7T25QS,Rajkumar Hirani,"Comedy, Drama",PG-13,prime video
6,103,tt0012349,The Kid,8.2,1921,https://www.amazon.com/gp/video/detail/B0010WF6K6,Charles Chaplin,"Comedy, Drama, Family",Passed,prime video
7,132,tt0363163,Downfall,8.2,2004,https://www.amazon.com/gp/video/detail/B07TJ26FVJ,Oliver Hirschbiegel,"Biography, Drama, History, War",R,prime video
8,134,tt0053291,Some Like It Hot,8.2,1959,https://www.amazon.com/gp/video/detail/B00477MY1K,Billy Wilder,"Comedy, Music, Romance",Not Rated,prime video
9,137,tt0057115,The Great Escape,8.2,1963,https://www.amazon.com/gp/video/detail/B000IZ0WQ4,John Sturges,"Adventure, Drama, History, Thriller, War",Not Rated,prime video


## Hulu

In [87]:
url = "https://reelgood.com/movies/source/hulu?filter-imdb_start=8"
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [88]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

movies = soup.find('table', class_="css-1179hly").find_all('td', class_="css-1u7zfla e126mwsw1")

In [89]:
titles = []
hulu_urls = []

for movie in movies:
    titles.append(movie.a.text)
    rg_url = movie.find('meta', itemprop="url")['content']
    response = requests.get(rg_url)
    rg_soup = BeautifulSoup(response.text, 'html.parser')
    hulu_urls.append(rg_soup.find('div',title="Stream on Hulu").a['href'].split('?')[0])
    print(movie.a.text +" - "+ rg_soup.find('div',title="Stream on Hulu").a['href'].split('?')[0])

The Dark Knight - https://www.hulu.com/watch/2ce6365a-46e7-43d9-8d77-5b84753a663c
GoodFellas - https://www.hulu.com/watch/01d5fb21-5896-420b-9346-9667bfc235d4
Parasite - https://www.hulu.com/watch/2fd691a0-f66b-467f-8635-00d7f151f3d4
The Green Mile - https://www.hulu.com/watch/e794ff67-db78-4ff0-b148-f251f3c1a171
Good Will Hunting - https://www.hulu.com/watch/good-will-hunting-0bf6e2a3-4a3f-40f3-b0c3-5e108ff8ffab
Batman Begins - https://www.hulu.com/watch/4dee2ca2-a81a-4988-8fe5-952637acbb5d
Grave of the Fireflies - https://www.hulu.com/watch/6887a224-9db8-4eb0-b052-b9f5d53789c6
Kill Bill: Vol. 1 - https://www.hulu.com/watch/91101089-b0d6-430e-b967-b18e81b416c0
Kill Bill: Vol. 2 - https://www.hulu.com/watch/d79041c1-dd27-42d5-ac13-1fad14f8d5e8
Warrior - https://www.hulu.com/watch/59b05290-3dc3-4a68-a37e-dae176ea4e12
Akira - https://www.hulu.com/watch/7a8680eb-a5fe-461d-a87f-998c278f624e
The Graduate - https://www.hulu.com/watch/bfe65adb-a6bb-4d05-b264-bba39aa7045a
Portrait of a Lady on

In [90]:
movie_items = soup.find('table', class_="css-1179hly").find_all('tr', class_="css-o6sgwe")

years = []

for movie in movie_items:
    td_rating = movie.find('td', class_="css-1u11l3y")
    years.append(td_rating.text)
    print(td_rating.text)

2008
1990
2019
1999
1997
2005
1988
2003
2004
2011
1988
1967
2019
2018
2013
2018
2019
2004
2019
2019
2019
2005
2018
2017
1972
1992
2017
2019
2009
2009
2019
2015
2019
2017


In [91]:
hulu_df = pd.DataFrame({"title": titles, "year": years, "hulu_url": hulu_urls})
hulu_df

,title,year,hulu_url
0,The Dark Knight,2008,https://www.hulu.com/watch/2ce6365a-46e7-43d9-...
1,GoodFellas,1990,https://www.hulu.com/watch/01d5fb21-5896-420b-...
2,Parasite,2019,https://www.hulu.com/watch/2fd691a0-f66b-467f-...
3,The Green Mile,1999,https://www.hulu.com/watch/e794ff67-db78-4ff0-...
4,Good Will Hunting,1997,https://www.hulu.com/watch/good-will-hunting-0...
5,Batman Begins,2005,https://www.hulu.com/watch/4dee2ca2-a81a-4988-...
6,Grave of the Fireflies,1988,https://www.hulu.com/watch/6887a224-9db8-4eb0-...
7,Kill Bill: Vol. 1,2003,https://www.hulu.com/watch/91101089-b0d6-430e-...
8,Kill Bill: Vol. 2,2004,https://www.hulu.com/watch/d79041c1-dd27-42d5-...
9,Warrior,2011,https://www.hulu.com/watch/59b05290-3dc3-4a68-...


In [100]:
browser.quit()

In [92]:
# Clean df
hulu_df = hulu_df[["title", "year", "hulu_url"]]
print(len(hulu_df))
hulu_df

34


,title,year,hulu_url
0,The Dark Knight,2008,https://www.hulu.com/watch/2ce6365a-46e7-43d9-...
1,GoodFellas,1990,https://www.hulu.com/watch/01d5fb21-5896-420b-...
2,Parasite,2019,https://www.hulu.com/watch/2fd691a0-f66b-467f-...
3,The Green Mile,1999,https://www.hulu.com/watch/e794ff67-db78-4ff0-...
4,Good Will Hunting,1997,https://www.hulu.com/watch/good-will-hunting-0...
5,Batman Begins,2005,https://www.hulu.com/watch/4dee2ca2-a81a-4988-...
6,Grave of the Fireflies,1988,https://www.hulu.com/watch/6887a224-9db8-4eb0-...
7,Kill Bill: Vol. 1,2003,https://www.hulu.com/watch/91101089-b0d6-430e-...
8,Kill Bill: Vol. 2,2004,https://www.hulu.com/watch/d79041c1-dd27-42d5-...
9,Warrior,2011,https://www.hulu.com/watch/59b05290-3dc3-4a68-...


In [123]:
# Match hulu and IMDB movies
imdb_hulu = pd.merge(imdb_df, hulu_df, on="title")
imdb_hulu = imdb_hulu[["rank", "id", "title", "rating", "year_x", "hulu_url", "director", "genre", "mpaa_rating"]]
imdb_hulu = imdb_hulu.rename(columns={"year_x":"year", "hulu_url":"url"})
imdb_hulu["service"] = "hulu"
imdb_hulu

,rank,id,title,rating,year,url,director,genre,mpaa_rating,service
0,4,tt0468569,The Dark Knight,9.0,2008,https://www.hulu.com/watch/2ce6365a-46e7-43d9-...,Christopher Nolan,"Action, Crime, Drama, Thriller",PG-13,hulu
1,27,tt6751668,Parasite,8.5,2019,https://www.hulu.com/watch/2fd691a0-f66b-467f-...,Bong Joon Ho,"Comedy, Drama, Thriller",R,hulu
2,29,tt0120689,The Green Mile,8.5,1999,https://www.hulu.com/watch/e794ff67-db78-4ff0-...,Frank Darabont,"Crime, Drama, Fantasy, Mystery",R,hulu
3,48,tt0095327,Grave of the Fireflies,8.4,1988,https://www.hulu.com/watch/6887a224-9db8-4eb0-...,Isao Takahata,"Animation, Drama, War",Not Rated,hulu
4,89,tt0119217,Good Will Hunting,8.3,1997,https://www.hulu.com/watch/good-will-hunting-0...,Gus Van Sant,"Drama, Romance",R,hulu
5,129,tt0372784,Batman Begins,8.2,2005,https://www.hulu.com/watch/4dee2ca2-a81a-4988-...,Christopher Nolan,"Action, Adventure",PG-13,hulu
6,160,tt1291584,Warrior,8.1,2011,https://www.hulu.com/watch/59b05290-3dc3-4a68-...,Gavin O'Connor,"Drama, Sport",PG-13,hulu
7,176,tt0266697,Kill Bill: Vol. 1,8.1,2003,https://www.hulu.com/watch/91101089-b0d6-430e-...,Quentin Tarantino,"Action, Crime, Thriller",R,hulu
8,232,tt8613070,Portrait of a Lady on Fire,8.0,2019,https://www.hulu.com/watch/d514447d-1954-4bf8-...,Céline Sciamma,"Drama, Romance",R,hulu


## Combine dataframe

In [148]:
movie_db = pd.concat([netflix_newdf, disney_newdf])
movie_db = pd.concat([movie_db, imdb_prime])
movie_db = pd.concat([movie_db, imdb_hulu])
movie_db = movie_db.drop_duplicates(subset="id", keep="first")
movie_db

,rank,id,title,rating,year,director,genre,mpaa_rating,service,url
0,6,tt0108052,Schindler's List,8.9,1993,Steven Spielberg,"Biography, Drama, History",R,netflix,Not Available
1,7,tt0167260,The Lord of the Rings: The Return of the King,8.9,2003,Peter Jackson,"Adventure, Drama, Fantasy",PG-13,netflix,Not Available
2,8,tt0110912,Pulp Fiction,8.8,1994,Quentin Tarantino,"Crime, Drama",R,netflix,Not Available
3,13,tt1375666,Inception,8.7,2010,Christopher Nolan,"Action, Adventure, Sci-Fi, Thriller",PG-13,netflix,https://www.netflix.com/watch/70131314
4,15,tt0167261,The Lord of the Rings: The Two Towers,8.7,2002,Peter Jackson,"Adventure, Drama, Fantasy",PG-13,netflix,Not Available
...,...,...,...,...,...,...,...,...,...,...
2,29,tt0120689,The Green Mile,8.5,1999,Frank Darabont,"Crime, Drama, Fantasy, Mystery",R,hulu,https://www.hulu.com/watch/e794ff67-db78-4ff0-...
3,48,tt0095327,Grave of the Fireflies,8.4,1988,Isao Takahata,"Animation, Drama, War",Not Rated,hulu,https://www.hulu.com/watch/6887a224-9db8-4eb0-...
4,89,tt0119217,Good Will Hunting,8.3,1997,Gus Van Sant,"Drama, Romance",R,hulu,https://www.hulu.com/watch/good-will-hunting-0...
5,129,tt0372784,Batman Begins,8.2,2005,Christopher Nolan,"Action, Adventure",PG-13,hulu,https://www.hulu.com/watch/4dee2ca2-a81a-4988-...


In [151]:
movies = movie_db[["rank", "id", "title", "rating", "year", "genre", "mpaa_rating"]]
movies = movies.rename(columns={"id":"title_id"})
movies

,rank,title_id,title,rating,year,genre,mpaa_rating
0,6,tt0108052,Schindler's List,8.9,1993,"Biography, Drama, History",R
1,7,tt0167260,The Lord of the Rings: The Return of the King,8.9,2003,"Adventure, Drama, Fantasy",PG-13
2,8,tt0110912,Pulp Fiction,8.8,1994,"Crime, Drama",R
3,13,tt1375666,Inception,8.7,2010,"Action, Adventure, Sci-Fi, Thriller",PG-13
4,15,tt0167261,The Lord of the Rings: The Two Towers,8.7,2002,"Adventure, Drama, Fantasy",PG-13
...,...,...,...,...,...,...,...
2,29,tt0120689,The Green Mile,8.5,1999,"Crime, Drama, Fantasy, Mystery",R
3,48,tt0095327,Grave of the Fireflies,8.4,1988,"Animation, Drama, War",Not Rated
4,89,tt0119217,Good Will Hunting,8.3,1997,"Drama, Romance",R
5,129,tt0372784,Batman Begins,8.2,2005,"Action, Adventure",PG-13


In [152]:
streaming_service = movie_db[["id", "service", "url"]]
streaming_service = streaming_service.rename(columns={"id":"title_id", "service":"streaming_service"})
streaming_service

,title_id,streaming_service,url
0,tt0108052,netflix,Not Available
1,tt0167260,netflix,Not Available
2,tt0110912,netflix,Not Available
3,tt1375666,netflix,https://www.netflix.com/watch/70131314
4,tt0167261,netflix,Not Available
...,...,...,...
2,tt0120689,hulu,https://www.hulu.com/watch/e794ff67-db78-4ff0-...
3,tt0095327,hulu,https://www.hulu.com/watch/6887a224-9db8-4eb0-...
4,tt0119217,hulu,https://www.hulu.com/watch/good-will-hunting-0...
5,tt0372784,hulu,https://www.hulu.com/watch/4dee2ca2-a81a-4988-...


# Load

In [153]:
# Connect to SQL
rds_connection_string = username+":"+password+"@localhost:5432/ETL_Project"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [154]:
engine.table_names()

['movies', 'streaming']

In [155]:
# Use pandas to load csv converted DataFrame into database
movies.to_sql(name='movies', con=engine, if_exists='append', index=False)

In [156]:
# Use pandas to load csv converted DataFrame into database
streaming_service.to_sql(name='streaming', con=engine, if_exists='append', index=False)

In [157]:
# Sample Query on movies table
pd.read_sql_query('select * from movies', con=engine).head()

,title_id,rank,title,rating,year,genre,mpaa_rating
0,tt0108052,6,Schindler's List,8.9,1993,"Biography, Drama, History",R
1,tt0167260,7,The Lord of the Rings: The Return of the King,8.9,2003,"Adventure, Drama, Fantasy",PG-13
2,tt0110912,8,Pulp Fiction,8.8,1994,"Crime, Drama",R
3,tt1375666,13,Inception,8.7,2010,"Action, Adventure, Sci-Fi, Thriller",PG-13
4,tt0167261,15,The Lord of the Rings: The Two Towers,8.7,2002,"Adventure, Drama, Fantasy",PG-13


In [158]:
# Sample Query on streaming table
pd.read_sql_query('select * from streaming', con=engine).head()

,title_id,streaming_service,url
0,tt0108052,netflix,Not Available
1,tt0167260,netflix,Not Available
2,tt0110912,netflix,Not Available
3,tt1375666,netflix,https://www.netflix.com/watch/70131314
4,tt0167261,netflix,Not Available
